# Week 2 Exercise Solution - Technical Q&A Chatbot

**Author:** Samuel Kalu  
**Team:** Euclid  
**Week:** 2

## Overview

This solution builds upon the Week 1 Exercise technical question/answerer, enhancing it with:
- ✅ Gradio UI for interactive chat
- ✅ Streaming responses for better UX
- ✅ System prompt customization for domain expertise
- ✅ Model switching (OpenAI, Anthropic, Ollama)
- ✅ **Bonus:** Tool integration for dynamic information retrieval

## Use Cases
- Language tutor
- Company onboarding solution
- Course companion AI
- Technical support assistant

In [ ]:
# Imports
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import requests

In [ ]:
# Load environment variables
load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

# Print key status for debugging
if openai_api_key:
    print(f"✓ OpenAI API Key: {openai_api_key[:8]}...")
else:
    print("✗ OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"✓ Anthropic API Key: {anthropic_api_key[:7]}...")
else:
    print("✗ Anthropic API Key not set")

if google_api_key:
    print(f"✓ Google API Key: {google_api_key[:8]}...")
else:
    print("✗ Google API Key not set")

## Model Configuration

Define available models for switching

In [ ]:
# Available models configuration
MODELS = {
    "OpenAI (gpt-4.1-mini)": {
        "provider": "openai",
        "model_name": "gpt-4.1-mini",
        "client": None
    },
    "OpenAI (gpt-4o)": {
        "provider": "openai",
        "model_name": "gpt-4o",
        "client": None
    },
    "Anthropic (Claude 3.5 Sonnet)": {
        "provider": "anthropic",
        "model_name": "claude-3-5-sonnet-20241022",
        "client": None
    },
    "Ollama (llama3.2)": {
        "provider": "ollama",
        "model_name": "llama3.2",
        "client": None
    }
}

# Initialize clients
if openai_api_key:
    MODELS["OpenAI (gpt-4.1-mini)"]["client"] = OpenAI(api_key=openai_api_key)
    MODELS["OpenAI (gpt-4o)"]["client"] = OpenAI(api_key=openai_api_key)

if anthropic_api_key:
    # Use OpenAI-compatible endpoint for Anthropic
    MODELS["Anthropic (Claude 3.5 Sonnet)"]["client"] = OpenAI(
        api_key=anthropic_api_key,
        base_url="https://api.anthropic.com/v1/"
    )

# Ollama (local)
try:
    MODELS["Ollama (llama3.2)"]["client"] = OpenAI(
        base_url="http://localhost:11434/v1",
        api_key="ollama"
    )
    print("✓ Ollama client initialized")
except:
    print("✗ Ollama not running locally")

## System Prompts for Different Domains

Pre-defined expert personas for different use cases

In [ ]:
SYSTEM_PROMPTS = {
    "General Assistant": "You are a helpful, knowledgeable assistant. Provide clear, accurate, and concise answers. If you don't know something, say so.",
    "Language Tutor": "You are an expert language tutor. Help students learn by explaining concepts clearly, providing examples, and correcting mistakes gently. Adapt to the student's level.",
    "Technical Expert": "You are a senior software engineer with expertise in modern technologies. Explain technical concepts clearly, provide code examples when helpful, and follow best practices.",
    "Course Companion": "You are an AI companion for this LLM Engineering course. Help students understand concepts, clarify doubts, and provide encouraging guidance. Reference course material when relevant.",
    "Company Onboarding": "You are an onboarding specialist for new employees. Provide clear information about company policies, procedures, and culture. Be welcoming and supportive."
}

## Bonus: Tool Integration

Add tool capabilities for dynamic information retrieval

In [ ]:
# Tool: Get current time/date
def get_current_time():
    """Get the current date and time"""
    from datetime import datetime
    return f"Current date/time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"

# Tool: Simple calculator
def calculate(expression: str):
    """Evaluate a mathematical expression"""
    try:
        result = eval(expression)
        return f"Result: {result}"
    except Exception as e:
        return f"Error: {str(e)}"

# Tool: Weather lookup (mock)
def get_weather(city: str):
    """Get current weather for a city (mock data)"""
    weather_data = {
        "london": "15°C, Partly Cloudy",
        "new york": "22°C, Sunny",
        "tokyo": "28°C, Clear",
        "paris": "18°C, Overcast",
        "sydney": "25°C, Sunny"
    }
    weather = weather_data.get(city.lower(), "Weather data not available for this city")
    return f"Current weather in {city.title()}: {weather}"

# Tool definitions for OpenAI API
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "get_current_time",
            "description": "Get the current date and time",
            "parameters": {
                "type": "object",
                "properties": {},
                "required": []
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "calculate",
            "description": "Evaluate a mathematical expression",
            "parameters": {
                "type": "object",
                "properties": {
                    "expression": {
                        "type": "string",
                        "description": "The mathematical expression to evaluate"
                    }
                },
                "required": ["expression"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get current weather for a city",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "The city name"
                    }
                },
                "required": ["city"]
            }
        }
    }
]

# Tool execution mapping
TOOL_FUNCTIONS = {
    "get_current_time": lambda: get_current_time(),
    "calculate": lambda args: calculate(args.get("expression", "")),
    "get_weather": lambda args: get_weather(args.get("city", ""))
}

## Chat Functions

Core chat logic with streaming and tool support

In [ ]:
def execute_tool(tool_call):
    """Execute a tool call and return the result"""
    function_name = tool_call.function.name
    arguments = json.loads(tool_call.function.arguments)
    
    if function_name == "get_current_time":
        return get_current_time()
    elif function_name == "calculate":
        return calculate(arguments.get("expression", ""))
    elif function_name == "get_weather":
        return get_weather(arguments.get("city", ""))
    
    return "Tool not found"


def chat_with_tools(message, history, system_prompt, model_choice, use_tools):
    """Main chat function with tool support"""
    model_config = MODELS.get(model_choice)
    
    if not model_config or not model_config["client"]:
        return f"Error: Model '{model_choice}' is not available. Please check your API keys or ensure Ollama is running."
    
    client = model_config["client"]
    model_name = model_config["model_name"]
    
    # Convert Gradio history to OpenAI format
    history_formatted = [{"role": h["role"], "content": h["content"]} for h in history]
    
    # Build messages
    messages = [{"role": "system", "content": system_prompt}] + history_formatted + [{"role": "user", "content": message}]
    
    try:
        # Make API call
        kwargs = {
            "model": model_name,
            "messages": messages
        }
        
        if use_tools and model_config["provider"] == "openai":
            kwargs["tools"] = TOOLS
        
        response = client.chat.completions.create(**kwargs)
        
        # Handle tool calls
        if use_tools and response.choices[0].finish_reason == "tool_calls":
            assistant_message = response.choices[0].message
            
            # Execute each tool call
            for tool_call in assistant_message.tool_calls:
                tool_result = execute_tool(tool_call)
                messages.append({
                    "role": "assistant",
                    "content": None,
                    "tool_calls": [tool_call]
                })
                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": tool_result
                })
            
            # Get final response with tool results
            response = client.chat.completions.create(
                model=model_name,
                messages=messages,
                tools=TOOLS
            )
        
        return response.choices[0].message.content
    
    except Exception as e:
        return f"Error: {str(e)}"


def chat_streaming(message, history, system_prompt, model_choice, use_tools):
    """Streaming version of chat function"""
    model_config = MODELS.get(model_choice)
    
    if not model_config or not model_config["client"]:
        yield f"Error: Model '{model_choice}' is not available."
        return
    
    client = model_config["client"]
    model_name = model_config["model_name"]
    
    # Convert history
    history_formatted = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = [{"role": "system", "content": system_prompt}] + history_formatted + [{"role": "user", "content": message}]
    
    try:
        kwargs = {
            "model": model_name,
            "messages": messages,
            "stream": True
        }
        
        if use_tools and model_config["provider"] == "openai":
            kwargs["tools"] = TOOLS
        
        full_response = ""
        for chunk in client.chat.completions.create(**kwargs):
            if chunk.choices[0].delta.content:
                content = chunk.choices[0].delta.content
                full_response += content
                yield full_response
    
    except Exception as e:
        yield f"Error: {str(e)}"

## Gradio UI

Create an interactive chat interface with all controls

In [ ]:
def create_chat_interface():
    """Create the Gradio chat interface"""
    
    with gr.Blocks(title="Technical Q&A Chatbot", theme=gr.themes.Soft()) as demo:
        gr.Markdown("""
        # 🤖 Technical Q&A Chatbot
        ### Week 2 Exercise Solution - Samuel Kalu (Team Euclid)
        
        An intelligent chatbot with:
        - Multiple model support
        - Expert system prompts
        - Tool integration (calculator, weather, time)
        - Streaming responses
        """)
        
        with gr.Row():
            with gr.Column(scale=3):
                # Chat interface
                chatbot = gr.Chatbot(
                    type="messages",
                    height=500,
                    placeholder="Ask me anything!"
                )
                
                with gr.Row():
                    msg_input = gr.Textbox(
                        placeholder="Type your message...",
                        scale=4,
                        show_label=False
                    )
                    send_btn = gr.Button("Send", variant="primary", scale=1)
                    clear_btn = gr.Button("Clear", scale=1)
                
            with gr.Column(scale=1):
                # Controls
                gr.Markdown("### ⚙️ Settings")
                
                model_dropdown = gr.Dropdown(
                    choices=list(MODELS.keys()),
                    value="OpenAI (gpt-4.1-mini)",
                    label="Model"
                )
                
                persona_dropdown = gr.Dropdown(
                    choices=list(SYSTEM_PROMPTS.keys()),
                    value="General Assistant",
                    label="AI Persona"
                )
                
                system_prompt_area = gr.Textbox(
                    value=SYSTEM_PROMPTS["General Assistant"],
                    label="System Prompt (Customizable)",
                    lines=4,
                    placeholder="Enter custom system prompt..."
                )
                
                use_tools_checkbox = gr.Checkbox(
                    label="Enable Tools\n(Calculator, Weather, Time)",
                    value=False
                )
                
                streaming_checkbox = gr.Checkbox(
                    label="Enable Streaming",
                    value=True
                )
        
        # Event handlers
        def update_system_prompt(persona):
            return SYSTEM_PROMPTS.get(persona, SYSTEM_PROMPTS["General Assistant"])
        
        persona_dropdown.change(
            fn=update_system_prompt,
            inputs=[persona_dropdown],
            outputs=[system_prompt_area]
        )
        
        def respond(message, history, system_prompt, model, use_tools, use_streaming):
            if use_streaming:
                response = ""
                for chunk in chat_streaming(message, history, system_prompt, model, use_tools):
                    response = chunk
                    yield response
            else:
                response = chat_with_tools(message, history, system_prompt, model, use_tools)
                yield response
        
        def user_message_submit(user_message, history):
            return "", history + [{"role": "user", "content": user_message}]
        
        def bot_response(history, system_prompt, model, use_tools, use_streaming):
            if not history or history[-1]["role"] != "user":
                return history
            
            user_message = history[-1]["content"]
            
            for response in respond(
                user_message, 
                history[:-1], 
                system_prompt, 
                model, 
                use_tools, 
                use_streaming
            ):
                history[-1] = {"role": "assistant", "content": response}
                yield history
        
        # Submit on button click
        send_btn.click(
            fn=user_message_submit,
            inputs=[msg_input, chatbot],
            outputs=[msg_input, chatbot]
        ).then(
            fn=bot_response,
            inputs=[chatbot, system_prompt_area, model_dropdown, use_tools_checkbox, streaming_checkbox],
            outputs=[chatbot]
        )
        
        # Submit on Enter
        msg_input.submit(
            fn=user_message_submit,
            inputs=[msg_input, chatbot],
            outputs=[msg_input, chatbot]
        ).then(
            fn=bot_response,
            inputs=[chatbot, system_prompt_area, model_dropdown, use_tools_checkbox, streaming_checkbox],
            outputs=[chatbot]
        )
        
        # Clear button
        clear_btn.click(fn=lambda: [], outputs=[chatbot])
        
        gr.Markdown("""
        ---
        **Try the tools:** Ask "What's 15 * 23?" or "What's the weather in London?" or "What time is it?"
        """)
    
    return demo

In [ ]:
# Launch the interface
if __name__ == "__main__":
    demo = create_chat_interface()
    demo.launch()
    
    # Alternative: Share publicly
    # demo.launch(share=True)

## Testing Section

Test individual components

In [ ]:
# Test tools
print("Testing Tools:")
print("=" * 40)
print(f"Time: {get_current_time()}")
print(f"Calculation: {calculate('2 + 2 * 5')}")
print(f"Weather: {get_weather('London')}")
print(f"Weather: {get_weather('Tokyo')}")

In [ ]:
# Test a simple model call
if MODELS["OpenAI (gpt-4.1-mini)"]["client"]:
    response = MODELS["OpenAI (gpt-4.1-mini)"]["client"].chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "Say hello in one sentence."}
        ]
    )
    print(f"\nModel Test Response: {response.choices[0].message.content}")

## Summary

### Features Implemented:
1. ✅ **Gradio UI** - Clean, interactive chat interface
2. ✅ **Streaming** - Real-time response generation
3. ✅ **System Prompts** - Customizable expert personas
4. ✅ **Model Switching** - OpenAI, Anthropic, Ollama support
5. ✅ **Tool Integration** - Calculator, Weather, Time tools

### Potential Enhancements:
- Audio input/output using `pydub`
- RAG integration with knowledge base
- Conversation history export
- Multi-language support
- Custom tool creation interface

### Lessons Learned:
- Gradio makes UI development incredibly fast
- Tool integration requires careful error handling
- Different models have different strengths for different tasks
- Streaming significantly improves user experience

---
**Built with ❤️ for LLM Engineering Bootcamp**